In [ ]:
import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.preprocessing import RobustScaler

import xgboost

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Загрузка выборки, генерация признака amount>0, очистка данных

In [ ]:
transactions = pd.read_csv('../data/raw/transactions.csv')
customers_gender = pd.read_csv('../data/raw/customers_gender_train.csv')

# Генерим важный признак для разделения на transactions[amount>0] и transactions[amount<0]
transactions['amount>0'] = transactions['amount'].apply(lambda x : int(x>0))

# (http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html)
robust_scaler = RobustScaler()
transactions['amount'] = robust_scaler.fit_transform(transactions['amount'])

Генерация признаков, связанных с временем

In [ ]:
# признаки для времени
transactions['day'] = transactions['tr_datetime'].apply(lambda s : int(s.split(' ')[0])) + 1
transactions['weekday'] = transactions['day'] % 7 + 1
transactions['date'] = transactions['tr_datetime'].apply(lambda s : s.split(' ')[1])
transactions['h_date'] = transactions['date'].apply(lambda t : t.split(':')[0])

del transactions['tr_datetime']

(*) Пачка из 13 групп признаков (различные статистики) для произвольного amount (не разделения на amount>0)

In [ ]:
# "Базовая фича" - Количество покупок по каждой категории

X = transactions.groupby('customer_id') \
                    .apply(lambda x: x[['mcc_code']].unstack().value_counts()) \
                    .unstack() \
                    .fillna(0)
X.rename(columns=lambda x: 'mcc_code_value_counts_'+str(x), inplace=True)

# Сумма покупок по каждой категории
F2 = transactions.groupby('customer_id').apply(lambda x : x.groupby('mcc_code')['amount'].sum()).unstack().fillna(0)
F2.rename(columns=lambda x: 'sum_'+str(x), inplace=True)

# Максимальная покупока по каждой категории

F3 = transactions.groupby('customer_id').apply(lambda x : x.groupby('mcc_code')['amount'].max()).unstack().fillna(0)
F3.rename(columns=lambda x: 'max_'+str(x), inplace=True)

# Дисперсия покупока по каждой категории

F4 = transactions.groupby('customer_id').apply(lambda x : x.groupby('mcc_code')['amount'].std()).unstack().fillna(0)
F4.rename(columns=lambda x: 'std_'+str(x), inplace=True)

# Средняя покупка по каждой категории

F5 = transactions.groupby('customer_id').apply(lambda x : x.groupby('mcc_code')['amount'].mean()).unstack().fillna(0)
F5.rename(columns=lambda x: 'mean_'+str(x), inplace=True)

# Количество tr_type
F6 = transactions.groupby('customer_id') \
                    .apply(lambda x: x[['tr_type']].unstack().value_counts()) \
                    .unstack() \
                    .fillna(0)
F6.rename(columns=lambda x: 'tr_type_value_counts_'+str(x), inplace=True)

# Сумма покупок по каждой tr_type
F7 = transactions.groupby('customer_id').apply(lambda x : x.groupby('tr_type')['amount'].sum()).unstack().fillna(0)
F7.rename(columns=lambda x: 'tr_type_sum_'+str(x), inplace=True)


# Max покупока по каждой tr_type
F8 = transactions.groupby('customer_id').apply(lambda x : x.groupby('tr_type')['amount'].max()).unstack().fillna(0)
F8.rename(columns=lambda x: 'tr_type_max_'+str(x), inplace=True)

# Std покупока по каждой tr_type
F9 = transactions.groupby('customer_id').apply(lambda x : x.groupby('tr_type')['amount'].std()).unstack().fillna(0)
F9.rename(columns=lambda x: 'tr_type_std_'+str(x), inplace=True)

# Mean покупока по каждой tr_type
F10 = transactions.groupby('customer_id').apply(lambda x : x.groupby('tr_type')['amount'].mean()).unstack().fillna(0)
F10.rename(columns=lambda x: 'tr_type_mean_'+str(x), inplace=True)

# Сколько дней клиент
F11 = pd.DataFrame({
        'customer_exp_days':transactions.groupby('customer_id')['day'].apply(lambda x : max(x) - min(x))
        })

# Количество покупок по дням недели
F12 = transactions.groupby('customer_id').apply(lambda x : x.groupby('weekday')['amount'].mean()).unstack().fillna(0)
F12.rename(columns=lambda x: 'mean_transactions_count_by_week_day_'+str(x), inplace=True)

# Транзакции по часам
F13 = transactions.groupby('customer_id').apply(lambda x : x.groupby('h_date')['amount'].mean()).unstack().fillna(0)
F13.rename(columns=lambda x: 'mean_transactions_count_by_h_'+str(x), inplace=True)

Выделяем транзакции с amount>0 и amount<0

In [ ]:
# выделяем на основе зануления
transactions_pos = transactions.copy()
transactions_pos['amount'] = transactions_pos['amount'] * transactions_pos['amount>0']

transactions_neg = transactions.copy()
transactions_neg['amount'] = transactions_neg['amount'] * transactions_neg['amount>0'].apply(lambda x : abs(x - 1))

Признаки (*) для transactions_pos и transactions_neg

In [ ]:
FP1 = transactions_pos.groupby('customer_id') \
                    .apply(lambda x: x[['mcc_code']].unstack().value_counts()) \
                    .unstack() \
                    .fillna(0)
FP1.rename(columns=lambda x: 'FP_mcc_code_value_counts_'+str(x), inplace=True)

# Сумма покупок по каждой категории
FP2 = transactions_pos.groupby('customer_id').apply(lambda x : x.groupby('mcc_code')['amount'].sum()).unstack().fillna(0)
FP2.rename(columns=lambda x: 'FP_sum_'+str(x), inplace=True)

# Максимальная покупока по каждой категории

FP3 = transactions_pos.groupby('customer_id').apply(lambda x : x.groupby('mcc_code')['amount'].max()).unstack().fillna(0)
FP3.rename(columns=lambda x: 'FP_max_'+str(x), inplace=True)

# Дисперсия покупока по каждой категории

FP4 = transactions_pos.groupby('customer_id').apply(lambda x : x.groupby('mcc_code')['amount'].std()).unstack().fillna(0)
FP4.rename(columns=lambda x: 'FP_std_'+str(x), inplace=True)

# Средняя покупка по каждой категории

FP5 = transactions_pos.groupby('customer_id').apply(lambda x : x.groupby('mcc_code')['amount'].mean()).unstack().fillna(0)
FP5.rename(columns=lambda x: 'FP_mean_'+str(x), inplace=True)

# Количество tr_type
FP6 = transactions_pos.groupby('customer_id') \
                    .apply(lambda x: x[['tr_type']].unstack().value_counts()) \
                    .unstack() \
                    .fillna(0)
FP6.rename(columns=lambda x: 'FP_tr_type_value_counts_'+str(x), inplace=True)

# Сумма покупок по каждой tr_type
FP7 = transactions_pos.groupby('customer_id').apply(lambda x : x.groupby('tr_type')['amount'].sum()).unstack().fillna(0)
FP7.rename(columns=lambda x: 'FP_tr_type_sum_'+str(x), inplace=True)


# Max покупока по каждой tr_type
FP8 = transactions_pos.groupby('customer_id').apply(lambda x : x.groupby('tr_type')['amount'].max()).unstack().fillna(0)
FP8.rename(columns=lambda x: 'FP_tr_type_max_'+str(x), inplace=True)

# Std покупока по каждой tr_type
FP9 = transactions_pos.groupby('customer_id').apply(lambda x : x.groupby('tr_type')['amount'].std()).unstack().fillna(0)
FP9.rename(columns=lambda x: 'FP_tr_type_std_'+str(x), inplace=True)

# Mean покупока по каждой tr_type
FP10 = transactions_pos.groupby('customer_id').apply(lambda x : x.groupby('tr_type')['amount'].mean()).unstack().fillna(0)
FP10.rename(columns=lambda x: 'FP_tr_type_mean_'+str(x), inplace=True)

# Сколько дней клиент
FP11 = pd.DataFrame({
        'FP_customer_exp_days':transactions_pos.groupby('customer_id')['day'].apply(lambda x : max(x) - min(x))
        })

# Количество покупок по дням недели
FP12 = transactions_pos.groupby('customer_id').apply(lambda x : x.groupby('weekday')['amount'].mean()).unstack().fillna(0)
FP12.rename(columns=lambda x: 'FP_mean_transactions_pos_count_by_week_day_'+str(x), inplace=True)

# Транзакции по часам
FP13 = transactions_pos.groupby('customer_id').apply(lambda x : x.groupby('h_date')['amount'].mean()).unstack().fillna(0)
FP13.rename(columns=lambda x: 'FP_mean_transactions_pos_count_by_h_'+str(x), inplace=True)

In [ ]:
FN1 = transactions_neg.groupby('customer_id') \
                    .apply(lambda x: x[['mcc_code']].unstack().value_counts()) \
                    .unstack() \
                    .fillna(0)
FN1.rename(columns=lambda x: 'FN_mcc_code_value_counts_'+str(x), inplace=True)

# Сумма покупок по каждой категории
FN2 = transactions_neg.groupby('customer_id').apply(lambda x : x.groupby('mcc_code')['amount'].sum()).unstack().fillna(0)
FN2.rename(columns=lambda x: 'FN_sum_'+str(x), inplace=True)

# Максимальная покупока по каждой категории

FN3 = transactions_neg.groupby('customer_id').apply(lambda x : x.groupby('mcc_code')['amount'].max()).unstack().fillna(0)
FN3.rename(columns=lambda x: 'FN_max_'+str(x), inplace=True)

# Дисперсия покупока по каждой категории

FN4 = transactions_neg.groupby('customer_id').apply(lambda x : x.groupby('mcc_code')['amount'].std()).unstack().fillna(0)
FN4.rename(columns=lambda x: 'FN_std_'+str(x), inplace=True)

# Средняя покупка по каждой категории

FN5 = transactions_neg.groupby('customer_id').apply(lambda x : x.groupby('mcc_code')['amount'].mean()).unstack().fillna(0)
FN5.rename(columns=lambda x: 'FN_mean_'+str(x), inplace=True)

# Количество tr_type
FN6 = transactions_neg.groupby('customer_id') \
                    .apply(lambda x: x[['tr_type']].unstack().value_counts()) \
                    .unstack() \
                    .fillna(0)
FN6.rename(columns=lambda x: 'FN_tr_type_value_counts_'+str(x), inplace=True)

# Сумма покупок по каждой tr_type
FN7 = transactions_neg.groupby('customer_id').apply(lambda x : x.groupby('tr_type')['amount'].sum()).unstack().fillna(0)
FN7.rename(columns=lambda x: 'FN_tr_type_sum_'+str(x), inplace=True)


# Max покупока по каждой tr_type
FN8 = transactions_neg.groupby('customer_id').apply(lambda x : x.groupby('tr_type')['amount'].max()).unstack().fillna(0)
FN8.rename(columns=lambda x: 'FN_tr_type_max_'+str(x), inplace=True)

# Std покупока по каждой tr_type
FN9 = transactions_neg.groupby('customer_id').apply(lambda x : x.groupby('tr_type')['amount'].std()).unstack().fillna(0)
FN9.rename(columns=lambda x: 'FN_tr_type_std_'+str(x), inplace=True)

# Mean покупока по каждой tr_type
FN10 = transactions_neg.groupby('customer_id').apply(lambda x : x.groupby('tr_type')['amount'].mean()).unstack().fillna(0)
FN10.rename(columns=lambda x: 'FN_tr_type_mean_'+str(x), inplace=True)

# Сколько дней клиент
FN11 = pd.DataFrame({
        'FN_customer_exp_days':transactions_neg.groupby('customer_id')['day'].apply(lambda x : max(x) - min(x))
        })

# Количество покупок по дням недели
FN12 = transactions_neg.groupby('customer_id').apply(lambda x : x.groupby('weekday')['amount'].mean()).unstack().fillna(0)
FN12.rename(columns=lambda x: 'FN_mean_transactions_neg_count_by_week_day_'+str(x), inplace=True)

# Транзакции по часам
FN13 = transactions_neg.groupby('customer_id').apply(lambda x : x.groupby('h_date')['amount'].mean()).unstack().fillna(0)
FN13.rename(columns=lambda x: 'FN_mean_transactions_neg_count_by_h_'+str(x), inplace=True)

Сливаем все группы признаков в один DataFrame

In [ ]:
XFeatured = pd.concat([X,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,FP1,FP2,FP3,FP4,FP5,FP6,FP7,FP8,FP9,FP10,FP11,FP12,FP13,FN1,FN2,FN3,FN4,FN5,FN6,FN7,FN8,FN9,FN10,FN11,FN12,FN13], axis=1, join='inner')
del X,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,FP1,FP2,FP3,FP4,FP5,FP6,FP7,FP8,FP9,FP10,FP11,FP12,FP13,FN1,FN2,FN3,FN4,FN5,FN6,FN7,FN8,FN9,FN10,FN11,FN12,FN13

In [ ]:
XFeatured.columns

Делим на train, test, target

In [ ]:
customers_gender = customers_gender.set_index('customer_id')
target = customers_gender.loc[XFeatured.index].gender
target = target.reset_index()

del target['customer_id']
target = target.dropna(0)
#target = target.values[:,0]

train = XFeatured.reset_index()
train = train.loc[target.index].set_index('customer_id')
test = XFeatured.drop(customers_gender.index)

del XFeatured

Можно сохранять промежуточные вычисления
```python
train.to_csv('../data/processed/train.csv', index=False)
test.to_csv('../data/processed/test.csv', index=False)
target.to_csv('../data/processed/target.csv', index=False)
```

Обернем наш классификатор в функцию ```apply_model```, которая подходит и для генерации submit, и для валидации

In [ ]:
def apply_model(tr, te, target):
    clf = xgboost.XGBClassifier(seed=0, learning_rate=0.02, max_depth=5, subsample=0.6815, colsample_bytree=0.701, n_estimators=1000, nthread=4)
    clf.fit(tr, target)
    
    return clf.predict_proba(te)[:, 1]

Делаем валидацию результата

In [ ]:
scores = []
N_SPLITS = 2
skf = StratifiedKFold(n_splits=N_SPLITS, random_state=0, shuffle=True)
for train_index, test_index in tqdm_notebook(skf.split(train.values, target.values[:,0]), total=N_SPLITS):
    X_train, X_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = target.iloc[train_index].values[:,0], target.iloc[test_index].values[:,0]
    
    scores += [ roc_auc_score(y_test, apply_model(X_train.copy(), X_test.copy(), y_train.copy())) ]
    
print np.mean(scores)

Делаем submit (в довесок сохраняем текущий блокнот + код функции apply_model)

In [ ]:
import time
import datetime
import inspect
from shutil import copyfile

submit_data = pd.DataFrame(test.index, columns=['customer_id'])
submit_data['gender'] = apply_model(train.copy(), test.copy(), target.copy())

ts = time.time()
ts = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')

submit_data.to_csv('../data/submits/sbm_' + ts + '.csv', index=False)
with open('../data/submits/sbm_' + ts + '.apply_model.py','w') as f:
    for line in inspect.getsourcelines(apply_model)[0]:
        f.write(line)
copyfile('task1.ipynb', '../data/submits/sbm_' + ts + '.notebook.ipynb')

Смешиваем наше решение с submit'ом товарища const (Konstantin Ivanin)

In [ ]:
from scipy.stats import rankdata


r_ans = pd.read_csv('../data/raw/task1_solution_by_const.csv')

blended_submit = submit_data.copy()
blended_submit['gender'] = rankdata(r_ans['gender'].values) + rankdata(submit_data['gender'].values)
blended_submit.to_csv('../data/submits/sbm_' + ts + '_blended.csv', index=False)